In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 1. Cheapest-to-Deliver Analysis

In [45]:
df_bonds = pd.read_excel('fut_bond_data_FVM5_2025-02-25.xlsx', sheet_name='bonds')
df_bonds.head()

,ticker,last_update_dt,px_last,maturity,days_to_mty,dur_adj_mid,cpn,nxt_cpn_dt,days_to_next_coupon,int_acc,accrued_days_between_cpn_dates,days_acc,basis_mid,repo_implied_reporate,repo_reporate,conversion
0,91282CLK Govt,2025-02-26,97.929688,2029-08-31,1646,4.031767,3.625,2025-02-28,1,1.802486,181,180,10.834997,4.069133,4.37,0.9135
1,91282CLN Govt,2025-02-26,97.343750,2029-09-30,1676,4.125978,3.500,2025-03-31,32,1.442308,182,150,13.850500,4.053931,4.37,0.9074
2,91282CLR Govt,2025-02-26,99.937500,2029-10-31,1707,4.153999,4.125,2025-04-30,62,1.356008,181,119,17.377000,4.074078,4.37,0.9293
3,91282CMA Govt,2025-02-26,99.960938,2029-11-30,1737,4.236733,4.125,2025-05-31,93,1.008585,182,89,22.374000,4.046093,4.37,0.9281
4,91282CMD Govt,2025-02-26,101.007812,2029-12-31,1768,4.298372,4.375,2025-06-30,123,0.700967,181,58,24.959000,4.077208,4.37,0.9367


In [46]:
df_future = pd.read_excel('fut_bond_data_FVM5_2025-02-25.xlsx', sheet_name='future')
# set field as index
df_future.set_index('field', inplace=True)
df_future

,FVM5 Comdty
field,
last_update_dt,2025-02-26 00:00:00
px_last,107.40625
last_tradeable_dt,2025-06-30 00:00:00
fut_dlv_dt_last,2025-07-03 00:00:00
fut_days_expire,124
fut_ctd,T 3.625 08/31/29
fut_ctd_px,97.929688
fut_ctd_gross_basis,-5.949538
fut_ctd_net_basis,1.543928


# 1.1)

In [47]:
df_bonds['Delivery Cost'] = df_bonds['px_last'] - df_bonds['conversion'] * df_future.loc['px_last', 'FVM5 Comdty']
#  bond with minimum delivery cost
ctd_bond = df_bonds.loc[df_bonds['Delivery Cost'].idxmin()]
ctd_bond['ticker']

'91282CLK Govt'

91282CLK Govt has the lowest delivery cost.

# 1.2)

In [48]:
df_bonds['Delivery Price Ratio'] = df_bonds['px_last'] / df_bonds['conversion']
ctd_bond = df_bonds.loc[df_bonds['Delivery Price Ratio'].idxmin()]
ctd_bond['ticker']

'91282CLK Govt'

Using clean prices and conversion factors, the T 3.625% Aug-2029 note minimizes the delivery cost and is therefore the cheapest-to-deliver for the June 2025 5-year Treasury future

# 1.3)

1. The bonds closest to being cheapest-to-deliver are the lower-coupon issues with shorter maturities, notably the 3.625% August 2029 and 3.50% September 2029 notes. 

2. In the current rate environment, with yields around 4–5%, low-coupon bonds are more likely to be CTD because the CBOT conversion factors are based on a notional 6% yield; when market yields are below 6%, high-coupon bonds trade at premiums that are not fully offset by their higher conversion factors. 

3. The CBOT uses 6% as the notional rate to create neutral delivery economics in a long-run average yield environment and to ensure stable and comparable futures pricing across deliverable bonds.

# 2. Implied Repo Rate

# 2.1)

In [49]:
tau = df_future.loc['fut_days_expire', 'FVM5 Comdty'] / 360.0


In [50]:
# Futures inputs
F = df_future.loc['px_last', 'FVM5 Comdty']  # futures price
tau = df_future.loc['fut_days_expire', 'FVM5 Comdty'] / 360  # ACT/360

# Semiannual coupon
df_bonds['cpn_semi'] = df_bonds['cpn'] / 2.0

# Accrued interest today
df_bonds['AI_today'] = df_bonds['int_acc']

# Accrued interest at delivery
df_bonds['days_acc_delivery'] = df_bonds['days_acc'] + df_future.loc['fut_days_expire', 'FVM5 Comdty']
df_bonds['AI_delivery'] = df_bonds['cpn_semi'] * df_bonds['days_acc_delivery'] / df_bonds['accrued_days_between_cpn_dates']

# Implied repo using your formula
df_bonds['implied_repo'] = (1 / tau) * ((F * df_bonds['conversion'] + df_bonds['AI_delivery']) / (df_bonds['px_last'] + df_bonds['AI_today']) - 1)

# Convert to percentage
df_bonds['implied_repo'] = df_bonds['implied_repo'] * 100

# Sort descending to find CTD
df_bonds = df_bonds.sort_values('implied_repo', ascending=False)

df_bonds[['ticker', 'implied_repo']]

,ticker,implied_repo
0,91282CLK Govt,4.155876
1,91282CLN Govt,3.846991
2,91282CLR Govt,3.691923
3,91282CMA Govt,3.243459
4,91282CMD Govt,3.134873
5,91282CMG Govt,2.736700


# 2.2)

In [51]:
# for each bond, calculate the repo spread using the CTD bond's implied repo rate and repo_reporate column
df_bonds['repo_spread'] = df_bonds['implied_repo'] - df_bonds['repo_reporate']
# bond with the highest implied repo rate
df_bonds.loc[df_bonds['implied_repo'].idxmax()][['ticker', 'repo_spread']]


ticker         91282CLK Govt
repo_spread        -0.214124
Name: 0, dtype: object

1. The CTD Bond - 91282CLK Govt has the highest implied repo.

2. The repo spread is negative, hence the implied repo is lower than the actual repo for the CTD bond.

3. If the implied repo rate (IRR) of a bond is lower than the actual market repo rate, it indicates that the bond futures contract is overpriced or "rich" relative to the cash bond. The futures price is too high compared to the cost of financing the bond, making it expensive to buy the bond and sell it via futures

# 2.3)

1. The implied repo rate (IRR) represents the annualized return from buying a bond at its dirty price, shorting the corresponding futures, and delivering the bond at expiration. If the IRR exceeds the actual repo rate, a cash-and-carry arbitrage opportunity exists: one can buy the bond, finance it at the actual repo rate, short the futures, and deliver the bond to lock in a risk-free profit. 

2. A bond can have a very high IRR but not be the cheapest-to-deliver (CTD) because the CTD is determined by the adjusted delivery cost after applying the conversion factor, which may favor another bond; in contrast, IRR depends on the bond’s price, coupons, and carry, so the bond with the highest IRR is not necessarily the one that minimizes delivery cost.

# 3. Conversion Factor Calculation

# 3.1)

In [52]:
# Convert coupon to decimal
df_bonds['cpn_decimal'] = df_bonds['cpn'] / 100

# Calculate semiannual periods
df_bonds['n'] = df_bonds['days_to_mty'] / 182.5
# Calculate CBOT conversion factor
df_bonds['phi_calc'] = (df_bonds['cpn_decimal']/0.06) * (1 - 1/(1.03**df_bonds['n'])) + 1/(1.03**df_bonds['n'])

# Compare with Bloomberg's conversion factor
df_bonds[['ticker', 'phi_calc', 'conversion']]

,ticker,phi_calc,conversion
0,91282CLK Govt,0.907368,0.9135
1,91282CLN Govt,0.900946,0.9074
2,91282CLR Govt,0.924516,0.9293
3,91282CMA Govt,0.923367,0.9281
4,91282CMD Govt,0.932561,0.9367
5,91282CMG Govt,0.926277,0.9307


# 3.2)

The **conversion factor (CF)** introduces a systematic bias because Treasury futures are standardized to a **6% notional coupon**. Bonds with actual coupons differing from 6% behave differently relative to the futures price:

1. **When rates are below 6%:**  
   - High-coupon bonds are relatively **overpriced** because their coupons are larger than the notional 6%.  
   - Low-coupon bonds are relatively **cheaper** after adjusting for the conversion factor.  
   - **CTD tends to be low-coupon bonds**.

2. **When rates are above 6%:**  
   - High-coupon bonds lose more value as their coupon is less attractive in a high-rate environment.  
   - Low-coupon bonds are relatively more valuable after conversion factor adjustment.  
   - **CTD tends to be high-coupon bonds**.

- The bonds have coupons in the range **3.5% to 4.375%**.  
- Current market rates are approximately **4%**, which is **below 6%**.  
- Therefore, **the lowest coupon bonds (3.5% and 3.625%) are expected to be closest to CTD**, as their conversion-factor-adjusted prices are lowest relative to the futures contract.

**Conclusion:**  
Given current rates below 6%, we expect **low-coupon bonds** in the dataset to be the **cheapest-to-deliver** into the Treasury futures contract.


# 4. Quality Option Value

# 4.1)

In [57]:
# calculate net basis
df_bonds['net_basis'] = df_bonds['px_last'] + df_bonds['int_acc'] - (df_bonds['conversion'] * F) 

In [58]:
# net basis for CTD bond
df_bonds.loc[df_bonds['Delivery Cost'].idxmin()]['net_basis']

np.float64(1.6165643128453127)

In [59]:
df_bonds[['ticker', 'net_basis']]

,ticker,net_basis
0,91282CLK Govt,1.616564
1,91282CLN Govt,1.325626
2,91282CLR Govt,1.480880
3,91282CMA Govt,1.285782
4,91282CMD Govt,1.101345
5,91282CMG Govt,0.822742


1. The Net_Basis for the **CTD bond (91282CLK)** is 1.616564

2. The **CTD bond is 91282CLK**, with a net basis of **1.616564**, which is the **highest among all deliverable bonds**.  
- All other deliverable bonds have **lower net basis values**, ranging from **0.822742 to 1.480880**.  
- This indicates that the **CTD bond is the most attractive for delivery**, because it maximizes the short’s potential gain relative to the futures price after adjusting for the conversion factor.  


# 4.2)

- A **positive net basis** of the CTD bond implies that the embedded options in the futures contract have **real value** to the short:

  1. **Quality option:** The short can choose the bond with the **highest net basis** (CTD) to maximize profit.  
  2. **Timing option:** The short can **deliver at the optimal time** during the delivery month.  
  3. **Wildcard option:** The short can decide **after trading closes** to take advantage of last-minute price movements.

- **Arbitrage does not drive net basis to zero** because of **transaction costs, financing costs, delivery timing constraints, and the optionality inherent in the futures contract**. These factors ensure the short retains some positive value from exercising these options.